Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## 1. Откроем файл с данными и изучим его 

In [2]:
try:
    df = pd.read_csv('C://Users/kuzmi/Downloads/users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')
   

In [3]:
df.head(5)

calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## 2. Разделим исходные данные на тестовую, валидационную и обучающую выборки

In [5]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Спрятанной тестовой выборки нет, поэтому разделяем данные в соотношении 3:1:1 (0.6, 0.2, 0.2)

### Обучающая выборка

In [6]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)

In [7]:
features_train.shape

(1928, 4)

In [8]:
target_train.shape

(1928,)

### Тестовая выборка

In [9]:
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)

In [10]:
features_test.shape

(643, 4)

In [11]:
target_test.shape

(643,)

### Валидационная выборка

In [12]:
features_valid.shape

(643, 4)

In [13]:
target_valid.shape

(643,)

## 3. Исследование качества моделей

### Дерево решений. Decision Tree Classifier

In [14]:
best_model_dtc = None
best_depth_dtc = 0
best_result_dtc = 0
for depth in range(1, 11):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth = depth).fit(features_train, target_train)
    predictions_valid_dtc = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(predictions_valid_dtc, target_valid)
    if best_result_dtc < result_dtc:
        best_model_dtc = model_dtc
        best_depth_dtc = depth
        best_result_dtc = result_dtc
print("Accurracy:", best_result_dtc, "Глубина дерева:", best_depth_dtc)       

Accurracy: 0.7853810264385692 Глубина дерева: 3


### Случайный лес. RandomForestClassifier

In [15]:
best_model_rfc = None
best_est_rfc = 0
best_depth_rfc = 0
best_result_rfc = 0
for est in tqdm(range(1, 11)):
    for depth in range(1, 11):
        model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth).fit(features_train, target_train)
        predictions_valid_rfc = model_rfc.predict(features_valid)
        result_rfc = accuracy_score(predictions_valid_rfc, target_valid)
        if best_result_rfc < result_rfc:
            best_model_rfc = model_rfc
            best_est_rfc = est
            best_depth_rfc = depth
            best_result_rfc = result_rfc
print("Accurracy:", best_result_rfc, "Глубина дерева:", best_depth_rfc, "Количество деревьев:", best_est_rfc)           

100%|██████████| 10/10 [00:01<00:00,  6.25it/s]

Accurracy: 0.80248833592535 Глубина дерева: 8 Количество деревьев: 8


### Логистическая регрессия. Logistic Regression

In [16]:
model_lr = LogisticRegression(random_state=12345).fit(features_train, target_train)
predictions_valid_lr = model_lr.predict(features_valid)
accuracy_lr = accuracy_score(predictions_valid_lr, target_valid)
print("Accuracy:", accuracy_lr,)

Accuracy: 0.7107309486780715


### Вывод


На данном этапе лучший результат у модели Случайный лес - Accuracy: 0.8

## 4. Проверка качества модели на тестовой выборке

In [17]:
model_rfc = RandomForestClassifier(random_state=12345, n_estimators = 8, max_depth = 8).fit(features_train, target_train)
predictions_test_rfc = model_rfc.predict(features_test)
result_test_rfc = accuracy_score(predictions_test_rfc, target_test)
print('Accuracy RandomForestClassifier:', result_test_rfc)

Accuracy RandomForestClassifier: 0.7962674961119751


Качество модели соотствует заданной метрике - Аccuracy больше 7.5

## 5. Проверка модели на вменяемость

In [18]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.7060653188180405


Accuracy DummyClassifier ниже, чем у нашей модели (0.7 к 0.796). Это позволяет нам сделать вывод, что наша модель вменяема и адекватна

## Общий вывод

В ходе исследования нами были созданы три модели машинного обучения: Дерево решений, Случайный лес и Логистическая регрессия. Каждая из этих моделей показала результат Accuracy равный 0.78, 0.8 и 0.71 соответсвенно. Лучший показатель - у модели Случайный лес, с ней мы продолжили работу. Дальнейшая проверка качества и проверка на вменяемость дали результат accuracy больше 7.5. Т.о. задачу считаем выполненной, рекомедуем использовать систему, построенную на модели Случайный лес.